In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import KFold

import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import roc_auc_score
from scipy.sparse import hstack



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv('../input/feedback-prize-english-language-learning/train.csv').fillna(' ')
test = pd.read_csv('../input/feedback-prize-english-language-learning/test.csv').fillna(' ')


In [ ]:
train["cohesion"].value_counts()

In [ ]:
class_cols = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']


In [ ]:
#for col in class_cols:
   # train[col] = train[col].astype("int")

In [ ]:
train.info()

In [ ]:
train_text = train['full_text']
test_text = test['full_text']
all_text = pd.concat([train_text, test_text])

In [ ]:
from warnings import filterwarnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, cross_validate
from sklearn.preprocessing import LabelEncoder
from textblob import Word, TextBlob
from wordcloud import WordCloud
from sklearn.metrics import accuracy_score  
filterwarnings("ignore")
from sklearn.feature_extraction.text import CountVectorizer 
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)
pd.set_option("display.float_format", lambda x:"%.2f" % 4)
pd.set_option("display.max_rows", None)

In [ ]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

In [ ]:
import nltk
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

In [ ]:
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

In [ ]:
test_features.shape

In [ ]:
train_features.shape

In [ ]:
#test_features = test_features.todense()

In [ ]:
#train_features = train_features.todense()

In [ ]:
columns = test.columns[1:]
columns

In [ ]:
from scipy.sparse import coo_matrix

In [ ]:
df_subm = pd.DataFrame(columns=["text_id",'cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions'])
df_subm["text_id"] = test["text_id"]
df_subm

In [ ]:
X = train_features
X_test = test_features
y = train[['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']].values

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_temp, y_train, y_temp = train_test_split(X,y,test_size=0.2, random_state=0)
X_valid,  X_test, y_valid, y_test = train_test_split(X_temp,y_temp,test_size=0.5, random_state=0)

In [ ]:
#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#from sklearn.linear_model import Ridge
#from sklearn.metrics import mean_squared_log_error
#from sklearn.model_selection import KFold
#import xgboost as xgb

#train_oof = np.zeros(target.shape)
#kf = KFold(n_splits=5, random_state=137, shuffle=True)

#for jj, (train_index, val_index) in enumerate(kf.split(X)):
   # print("Fitting fold", jj+1)
   # train_x = X[train_index]
   # val_x = X[val_index]
   # train_target = target[train_index]
   # val_target = target[val_index]
   # model = xgb.XGBRegressor(tree_method='hist')
   # model.fit(train_x, train_target)
   ## val_preds = np.clip(model.predict(val_x), 0, None)
   # train_oof[val_index] = val_preds
   # print(CRMSLE(val_target, val_preds)))

In [ ]:
X_train.shape

In [ ]:

from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor

model_lgbm = MultiOutputRegressor(LGBMRegressor(boosting_type = "dart",colsample_bytree=0.9, num_iterations=1000, random_state=19, verbose=-1))

model_lgbm.fit(X_train, y_train)
    


In [ ]:
pred = model_lgbm.predict(test_features)
preds = pd.DataFrame(pred, columns=class_cols)
submit_df = pd.concat([test[['text_id']], preds], axis=1)
submit_df.to_csv("submission.csv", index=False)